In [154]:
# linear algebra
import numpy as np
# data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
# for dimensionality reduction
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split


# Data PreProcessing

In [138]:
df_train = pd.read_csv('train.csv')
# let us understand the data


# Step6: Read the test.csv data

df_test = pd.read_csv('test.csv')

In [134]:
print(df_train.shape)
print("---------------------")
print(df_train.head())

(4209, 378)
---------------------
   ID       y  X0 X1  X2 X3 X4 X5 X6 X8  ...  X375  X376  X377  X378  X379  \
0   0  130.81   k  v  at  a  d  u  j  o  ...     0     0     1     0     0   
1   6   88.53   k  t  av  e  d  y  l  o  ...     1     0     0     0     0   
2   7   76.26  az  w   n  c  d  x  j  x  ...     0     0     0     0     0   
3   9   80.62  az  t   n  f  d  x  l  e  ...     0     0     0     0     0   
4  13   78.02  az  v   n  f  d  h  d  n  ...     0     0     0     0     0   

   X380  X382  X383  X384  X385  
0     0     0     0     0     0  
1     0     0     0     0     0  
2     0     1     0     0     0  
3     0     0     0     0     0  
4     0     0     0     0     0  

[5 rows x 378 columns]


In [135]:
y_train = df_train['y'].values

In [136]:
# iterate through all the columns which has X in the name of the column
cols = [c for c in df_train.columns if 'X' in c]
print('Number of features: {}'.format(len(cols)))

print('Feature types:')
df_train[cols].dtypes.value_counts()


Number of features: 376
Feature types:


int64     368
object      8
dtype: int64

In [ ]:
c = list()
for i in range(len(df.columns)):
    if(len(np.unique(df.iloc[:,i])) == 1):
        c.append(df.columns[i])
        
df.drop(c,axis = 1, inplace = True)
test.drop(c,axis = 1, inplace = True)

In [137]:
#Count the data in each of the columns

counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)

print('Constant features: {} Binary features: {} Categorical features: {}\n'
      .format(*[len(c) for c in counts]))
print('Constant features:', counts[0])
print('Categorical features:', counts[2])

Constant features: 12 Binary features: 356 Categorical features: 8

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']
Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [139]:
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))
y_train = df_train['y'].values
id_test = df_test['ID'].values

x_train = df_train[usable_columns]
x_test = df_test[usable_columns]


### Check for null and unique values for test and train sets.


In [142]:
#heck for null and unique values for test and train sets

def check_missing_values(df):
    if df.isnull().any().any():
        print("Missing Value")
    else:
        print("N no missing values")
check_missing_values(x_train)
check_missing_values(x_test)


N no missing values
N no missing values


In [170]:
##df_train.ID.unique()

array([   0,    6,    7, ..., 8412, 8415, 8417], dtype=int64)

### If for any column(s), the variance is equal to zero,  then you need to remove those variable(s).Apply label encoder

In [143]:


for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

,X275,X271,X338,X295,X242,X29,X157,X247,X83,X13,...,X209,X1,X298,X86,X125,X313,X385,X42,X370,X384
0,1,0,0,0,0,0,0,0,0,1,...,1,118,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,1,116,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,1,0,0,0,...,1,119,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,1,0,0,0,...,1,116,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,1,0,0,0,...,1,118,0,0,0,0,0,0,0,0


In [144]:


print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

### Perform dimensionality reduction.

In [145]:

n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)


In [152]:
print("actual shape:" ,x_train.shape)
print("after Dimensionality Reduction :", pca2_results_train.shape)

actual shape: (4209, 376)
after Dimensionality Reduction : (4209, 12)


In [153]:
print("actual shape:" ,x_test.shape)
print("after Dimensionality Reduction:", pca2_results_test.shape)

actual shape: (4209, 376)
after Dimensionality Reduction: (4209, 12)


### Predict your test_df values using XGBoost.

In [155]:
x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)

In [156]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4


In [157]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [158]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[10:21:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.0.0/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14834	valid-rmse:98.26296	train-r2:-58.35295	valid-r2:-67.63754
Multiple eval metrics have been passed: 'valid-r2' will be used for early stopping.

Will train until valid-r2 hasn't improved in 50 rounds.
[10]	train-rmse:81.27653	valid-rmse:80.36433	train-r2:-38.88428	valid-r2:-44.91014
[20]	train-rmse:66.71610	valid-rmse:65.77335	train-r2:-25.87403	valid-r2:-29.75260
[30]	train-rmse:54.86912	valid-rmse:53.89136	train-r2:-17.17722	valid-r2:-19.64525
[40]	train-rmse:45.24709	valid-rmse:44.22323	train-r2:-11.36097	valid-r2:-12.90218
[50]	train-rmse:37.44854	valid-rmse:36.37614	train-r2:-7.46723	valid-r2:-8.40622
[60]	train-rmse:31.14584	valid-rmse:30.02252	train-r2:-4.85695	valid-r2:-5.40732
[70]	train-rmse:26.08417	valid-rmse:24.91497	train-r2:-3.10795	valid-r2:-3.41268
[80]	train-rmse:22.04312	

### Predict your test_df values using xgboost

In [160]:
Predict your test_df values using xgboost

p_test = clf.predict(d_test)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

sub.head()

,ID,y
0,1,83.041672
1,2,97.248413
2,3,83.035416
3,4,76.871239
4,5,112.502762
